## Step 1: Install Dependencies

In [1]:
# Install required packages using pip magic command (best for Jupyter)
%pip install -q gradio openai python-dotenv

print("✅ All packages installed!")

/Users/retlawair/Desktop/deploying-ai/deploying-ai-env/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
✅ All packages installed!


## Step 2: Load Environment Variables

In [2]:
%load_ext dotenv
%dotenv ./.secrets

import os
api_key = os.getenv('API_GATEWAY_KEY')
print(f"✅ API Key loaded: {api_key[:3] if api_key else 'NOT FOUND'}...")

✅ API Key loaded: YTL...


## Step 3: Import Libraries

In [3]:
import gradio as gr
from openai import OpenAI
import os
from datetime import datetime

print(f"✅ Gradio {gr.__version__}")
print("✅ OpenAI SDK ready")

✅ Gradio 5.49.1
✅ OpenAI SDK ready


In [4]:
from pydantic import BaseModel, Field, field_validator
from typing import Optional, List, Literal
from enum import Enum

print("✅ Pydantic models ready")

✅ Pydantic models ready


## Step 4: Initialize OpenAI Client

In [5]:
# Initialize OpenAI client
client = OpenAI(
    default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')},
    base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1'
)

print("✅ OpenAI client initialized")

✅ OpenAI client initialized


In [ ]:
# Define allowed values as enums for type safety
class ToneType(str, Enum):
    FORMAL = "formal"
    CASUAL = "casual"
    FRIENDLY = "friendly"
    PROFESSIONAL = "professional"
    HUMOROUS = "humorous"

class StyleType(str, Enum):
    CONCISE = "concise"
    DETAILED = "detailed"
    CREATIVE = "creative"
    ANALYTICAL = "analytical"
    SOCRATIC = "socratic"

class RoleType(str, Enum):
    ASSISTANT = "helpful AI assistant"
    MENTOR = "mentor and educator"
    EXPERT = "subject matter expert"
    BRAINSTORMER = "creative brainstorming partner"
    THERAPIST = "empathetic listener"
    ANALYST = "data analyst"

# Core configuration model
class ChatConfig(BaseModel):
    """Pydantic model for chat personality and behavior configuration"""
    
    tone: ToneType = Field(
        default=ToneType.FRIENDLY,
        description="The tone of voice for responses"
    )
    conversational_style: StyleType = Field(
        default=StyleType.CONCISE,
        description="How detailed or creative responses should be"
    )
    role: RoleType = Field(
        default=RoleType.ASSISTANT,
        description="The character role the AI adopts"
    )
    temperature: float = Field(
        default=0.7,
        ge=0.0,
        le=2.0,
        description="Controls randomness (0=deterministic, 2=very creative)"
    )
    max_tokens: int = Field(
        default=1000,
        ge=100,
        le=4000,
        description="Maximum response length"
    )
    custom_system_prompt: Optional[str] = Field(
        default=None,
        description="Override default system prompt"
    )
    model: str = Field(
        default="gpt-4o-mini",
        description="OpenAI model to use"
    )
    
    @field_validator('temperature')
    @classmethod
    def validate_temperature(cls, v):
        if v < 0 or v > 2:
            raise ValueError('Temperature must be between 0 and 2')
        return v
    
    # this is a validator to ensure that the temperature value is within the acceptable range for OpenAI models. If a user tries to set a temperature outside of 0 to 2, it will raise a ValueError with a clear message.
    # guidelines are also soft enforced in the system prompt to ensure that even if a user tries to bypass the validator, the model will still be instructed not to respond to certain topics or reveal system instructions.

    def get_system_prompt(self) -> str:
        """Generate comprehensive system prompt based on configuration"""
        if self.custom_system_prompt:
            return self.custom_system_prompt
        
        return f"""You are a {self.role.value} with a {self.tone.value} tone.
Your conversational style is {self.conversational_style.value}.

Guidelines:
- Do not answer any requests on the following topics: Cats, Dogs, Horoscopes, Zodiac Signs, or Taylor Swift.
- Maintain consistency with your assigned tone throughout
- Adapt your response length based on your style (concise = short, detailed = thorough)
- Stay in character as a {self.role.value}
- Be helpful, accurate, and respectful"""
    
    class Config:
        use_enum_values = True

# Message model for type-safe message handling
class Message(BaseModel):
    role: Literal["system", "user", "assistant"] 
    content: str

# Chat session model to track state
class ChatSession(BaseModel):
    config: ChatConfig
    messages: List[Message] = Field(default_factory=list)
    
    def add_message(self, role: str, content: str):
        """Add message to session"""
        self.messages.append(Message(role=role, content=content))
    
    def get_messages_for_api(self) -> List[dict]:
        """Convert messages to OpenAI API format"""
        return [{"role": msg.role, "content": msg.content} for msg in self.messages]

# Predefined personality configurations
class PersonalityLibrary:
    """Collection of pre-configured chat personalities"""
    
    DEFAULT = ChatConfig(
        tone=ToneType.FRIENDLY,
        conversational_style=StyleType.CONCISE,
        role=RoleType.ASSISTANT,
        temperature=0.7
    )
    
    MENTOR = ChatConfig(
        tone=ToneType.PROFESSIONAL,
        conversational_style=StyleType.DETAILED,
        role=RoleType.MENTOR,
        temperature=0.6
    )
    
    CREATIVE = ChatConfig(
        tone=ToneType.CASUAL,
        conversational_style=StyleType.CREATIVE,
        role=RoleType.BRAINSTORMER,
        temperature=0.9
    )
    
    ANALYTICAL = ChatConfig(
        tone=ToneType.FORMAL,
        conversational_style=StyleType.ANALYTICAL,
        role=RoleType.ANALYST,
        temperature=0.5
    )
    
    @staticmethod
    def get_all_personalities():
        """Return all available personalities"""
        return {
            "default": PersonalityLibrary.DEFAULT,
            "mentor": PersonalityLibrary.MENTOR,
            "creative": PersonalityLibrary.CREATIVE,
            "analytical": PersonalityLibrary.ANALYTICAL
        }

print("✅ Pydantic models and PersonalityLibrary defined")

✅ Pydantic models and PersonalityLibrary defined


## Step 4.5: Define Chat Config Guardrails

In [7]:
import re
from typing import Dict, List, Tuple
from dataclasses import dataclass, field
import time

@dataclass
class GuardrailConfig:
    """Enterprise configuration for content guardrails"""
    forbidden_topics: List[str] = field(default_factory=lambda: [
        "cats", "dogs", "horoscopes", "zodiac signs", "taylor swift"
    ])
    forbidden_patterns: List[str] = field(default_factory=lambda: [
        r"\b(cats?|dogs?|horoscope|zodiac|taylor\s+swift)\b",
        r"prompt\s+injection", r"system\s+prompt", r"ignore\s+instructions"
    ])
    max_input_length: int = 2000
    min_input_length: int = 1
    max_tokens_per_request: int = 4000
    max_requests_per_minute: int = 60
    max_output_length: int = 4000
    min_output_length: int = 1

class GuardrailViolation(Exception):
    """Custom exception for guardrail violations"""
    def __init__(self, message: str, severity: str, violation_type: str):
        self.message = message
        self.severity = severity
        self.violation_type = violation_type
        super().__init__(f"[{severity}] {violation_type}: {message}")

class ContentGuardrails:
    """Enterprise-grade content validation"""
    def __init__(self, config: GuardrailConfig = None):
        self.config = config or GuardrailConfig()
        self.request_history: Dict[str, List[float]] = {}
        self.compiled_patterns = [re.compile(p, re.IGNORECASE) for p in self.config.forbidden_patterns]
    
    def validate_input(self, message: str, user_id: str = "default") -> Tuple[bool, str]:
        if not message or not isinstance(message, str):
            raise GuardrailViolation("Input must be a non-empty string", "CRITICAL", "invalid_input_type")
        if len(message) < self.config.min_input_length:
            raise GuardrailViolation(f"Input too short", "WARNING", "length_violation")
        if len(message) > self.config.max_input_length:
            raise GuardrailViolation(f"Input exceeds max length ({self.config.max_input_length})", "CRITICAL", "length_violation")
        
        message_lower = message.lower()
        for topic in self.config.forbidden_topics:
            if topic in message_lower:
                raise GuardrailViolation(f"Forbidden topic: '{topic}'", "CRITICAL", "forbidden_content")
        
        for pattern in self.compiled_patterns:
            if pattern.search(message):
                raise GuardrailViolation("Suspicious patterns detected", "CRITICAL", "prompt_injection")
        
        current_time = time.time()
        if user_id not in self.request_history:
            self.request_history[user_id] = []
        self.request_history[user_id] = [t for t in self.request_history[user_id] if current_time - t < 60]
        if len(self.request_history[user_id]) >= self.config.max_requests_per_minute:
            raise GuardrailViolation(f"Rate limit exceeded", "WARNING", "rate_limit")
        self.request_history[user_id].append(current_time)
        return True, "Input validation passed"
    
    def validate_output(self, response: str) -> Tuple[bool, str]:
        if not isinstance(response, str):
            raise GuardrailViolation("Output must be a string", "CRITICAL", "invalid_output_type")
        if len(response) < self.config.min_output_length:
            raise GuardrailViolation("Insufficient output", "WARNING", "output_length_violation")
        if len(response) > self.config.max_output_length:
            return True, response[:self.config.max_output_length] + "..."
        
        response_lower = response.lower()
        for topic in self.config.forbidden_topics:
            if topic in response_lower:
                raise GuardrailViolation(f"Forbidden content in output: '{topic}'", "CRITICAL", "forbidden_output_content")
        
        system_indicators = ["system prompt", "you are a", "role:", "guideline:", "instruction:"]
        for indicator in system_indicators:
            if indicator in response_lower and ("system" in response_lower or "instruction" in response_lower):
                raise GuardrailViolation("System prompt leakage detected", "CRITICAL", "prompt_leakage")
        return True, response

guardrails_config = GuardrailConfig()
guardrails = ContentGuardrails(guardrails_config)
print("✅ Enterprise guardrails system initialized")
print(f"   📋 Forbidden topics: {len(guardrails_config.forbidden_topics)}")
print(f"   🔍 Pattern checks: {len(guardrails_config.forbidden_patterns)}")
print(f"   ⏱️  Max input: {guardrails_config.max_input_length} chars")
print(f"   🚀 Rate limit: {guardrails_config.max_requests_per_minute} req/min")

✅ Enterprise guardrails system initialized
   📋 Forbidden topics: 5
   🔍 Pattern checks: 4
   ⏱️  Max input: 2000 chars
   🚀 Rate limit: 60 req/min


## Step 5: Define Chat Function

In [8]:
def chat_with_gpt(message, history, config=None):
    """
    Enhanced chat with GPT-4 using Pydantic configuration management.
    Maintains conversation history with configurable personality.
    INCLUDES ENTERPRISE-GRADE GUARDRAILS.
    
    Args:
        message: Current user message (string)
        history: List of message dictionaries with 'role' and 'content' keys
        config: ChatConfig Pydantic model (uses DEFAULT if None)
    
    Returns:
        Assistant's response (string)
    """
    if config is None:
        config = PersonalityLibrary.DEFAULT
    
    try:
        # ===== GUARDRAIL 1: INPUT VALIDATION (Hard Block) =====
        try:
            guardrails.validate_input(message)
        except GuardrailViolation as gv:
            if gv.severity == "CRITICAL":
                return f"⛔ Request blocked: {gv.message}"
            else:
                print(f"⚠️  {gv.message}")
        
        # Validate and ensure config is ChatConfig instance
        if not isinstance(config, ChatConfig):
            config = ChatConfig(**config) if isinstance(config, dict) else PersonalityLibrary.DEFAULT
        
        # Validate config parameters (hard limits)
        if config.temperature < 0 or config.temperature > 2:
            raise GuardrailViolation(
                "Temperature out of bounds",
                "CRITICAL",
                "parameter_violation"
            )
        
        if config.max_tokens < 100 or config.max_tokens > 4000:
            raise GuardrailViolation(
                "Max tokens out of bounds",
                "CRITICAL",
                "parameter_violation"
            )
        
        # Build messages list with system prompt and history
        messages = [
            {
                "role": "system",
                "content": config.get_system_prompt()
            }
        ]
        
        # Add conversation history (already in correct format)
        if history:
            messages.extend(history)
        
        # Add current message
        messages.append({"role": "user", "content": message})
        
        # Call OpenAI API with validated config
        response = client.chat.completions.create(
            model=config.model,
            messages=messages,
            temperature=config.temperature,
            max_tokens=config.max_tokens
        )
        
        assistant_response = response.choices[0].message.content
        
        # ===== GUARDRAIL 2: OUTPUT VALIDATION (Hard Block) =====
        try:
            is_valid, result = guardrails.validate_output(assistant_response)
            if is_valid:
                return result
        except GuardrailViolation as gv:
            if gv.severity == "CRITICAL":
                return "⛔ Response blocked by content policy"
            else:
                return assistant_response
        
        return assistant_response
    
    except GuardrailViolation as gv:
        return f"⛔ [{gv.severity}] {gv.message}"
    except Exception as e:
        return f"Error: {str(e)}"

print("✅ Enhanced chat function defined with Pydantic config")

✅ Enhanced chat function defined with Pydantic config


## Step 6: Create Gradio Interface

In [9]:
# Store config choices
config_choices = {
    "personality": "default",
    "tone": "friendly", 
    "style": "concise",
    "temperature": 0.7
}

def simple_chat(message, history):
    """Simple chat function with enterprise guardrails enforcement"""
    try:
        # ===== GUARDRAIL 1: INPUT VALIDATION (Hard Block) =====
        try:
            guardrails.validate_input(message, user_id="gradio_user")
        except GuardrailViolation as gv:
            return f"⛔ Request blocked: {gv.message}"
        
        # ===== GUARDRAIL 3: HARD LIMITS CHECK =====
        # Validate temperature is within bounds
        temp = config_choices.get("temperature", 0.7)
        if not (0.0 <= temp <= 2.0):
            return "⛔ Temperature out of valid range (0.0-2.0)"
        
        # Get base config from personality
        personalities = PersonalityLibrary.get_all_personalities()
        base_config = personalities.get(config_choices["personality"], PersonalityLibrary.DEFAULT)
        
        # Use selected tone and style directly as strings
        tone_str = config_choices["tone"]
        style_str = config_choices["style"]
        
        # Create simple system prompt with selected attributes
        system_prompt = f"""You are a helpful AI assistant with a {tone_str} tone.
Your conversational style is {style_str}.

Guidelines:
- Do not answer any requests on the following topics: Cats, Dogs, Horoscopes, Zodiac Signs, or Taylor Swift.
- Be helpful, accurate, and respectful."""
        
        # Build messages
        messages = [{"role": "system", "content": system_prompt}]
        if history:
            messages.extend(history)
        messages.append({"role": "user", "content": message})
        
        # Call API
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=temp,
            max_tokens=1000
        )
        
        assistant_response = response.choices[0].message.content
        
        # ===== GUARDRAIL 2: OUTPUT VALIDATION (Hard Block) =====
        try:
            is_valid, result = guardrails.validate_output(assistant_response)
            return result if is_valid else assistant_response
        except GuardrailViolation as gv:
            return "⛔ Response blocked by content policy"
    
    except GuardrailViolation as gv:
        return f"⛔ [{gv.severity}] {gv.message}"
    except Exception as e:
        return f"Error: {str(e)}"


def run_ai_risk_search(query, top_k=5, lexical_k=50, alpha=0.35):
    """Run hybrid lexical + semantic search against the AI Risk Database."""
    if not query or not query.strip():
        return "❌ Please enter a question.", []
    results = hybrid_search(query, top_k=top_k, lexical_k=lexical_k, alpha=alpha)
    markdown = format_search_results_markdown(results, query)
    return markdown, results


def landing_ai_risk_search(query, top_k=5):
    """Landing-page helper that also populates the AI Risk Search tab."""
    if not query or not query.strip():
        return "❌ Please enter a question.", "", "❌ Please enter a question.", []
    results = hybrid_search(query, top_k=top_k)
    markdown = format_search_results_markdown(results, query)
    landing_message = f"✅ Results loaded in **AI Risk Search** tab.\n\n{markdown}"
    return landing_message, query, markdown, results


with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 Walter's AI Chatbot & Security Scanner")
    gr.Markdown("Customize your chat experience or scan files for security threats")

    gr.Markdown("### 🔎 Quick Search the AI Risk Database")
    with gr.Row():
        landing_query = gr.Textbox(
            label="Ask about AI risks",
            placeholder="e.g., What are risks related to misinformation?"
        )
        landing_top_k = gr.Slider(
            minimum=3,
            maximum=10,
            value=5,
            step=1,
            label="Top results"
        )
        landing_search_btn = gr.Button("Search", variant="secondary")
    landing_output = gr.Markdown("")
    
    with gr.Tabs():
        # ===== TAB 1: Chat Interface =====
        with gr.Tab("💬 Chat"):
            with gr.Row():
                personality = gr.Dropdown(
                    choices=list(PersonalityLibrary.get_all_personalities().keys()),
                    value="default",
                    label="🎭 Personality"
                )
                tone = gr.Dropdown(
                    choices=[t.value for t in ToneType],
                    value="friendly",
                    label="🎵 Tone"
                )
                style = gr.Dropdown(
                    choices=[s.value for s in StyleType],
                    value="concise",
                    label="📝 Style"
                )
                temperature = gr.Slider(
                    minimum=0.0,
                    maximum=2.0,
                    value=0.7,
                    step=0.1,
                    label="🔥 Temperature"
                )
            
            personality.change(lambda x: config_choices.update({"personality": x}), inputs=personality)
            tone.change(lambda x: config_choices.update({"tone": x}), inputs=tone)
            style.change(lambda x: config_choices.update({"style": x}), inputs=style)
            temperature.change(lambda x: config_choices.update({"temperature": x}), inputs=temperature)
            
            chatbot = gr.ChatInterface(
                fn=simple_chat,
                examples=[
                    "What is machine learning?",
                    "Explain quantum computing",
                    "How do neural networks work?",
                    "What is the capital of France?"
                ],
                type="messages"
            )
        
        # ===== TAB 2: File Scanner =====
        with gr.Tab("🔒 File Scanner"):
            gr.Markdown("### VirusTotal File Security Scan with AI Interpretation")
            gr.Markdown("Upload a file to scan it for security threats. Results will be analyzed and explained by AI.")
            
            with gr.Row():
                file_input = gr.File(
                    label="📁 Select File to Scan",
                    type="filepath"
                )
                scan_button = gr.Button("🔍 Scan & Analyze", variant="primary", size="lg")
            
            with gr.Row():
                with gr.Column():
                    ai_interpretation = gr.Markdown("Upload a file and click 'Scan & Analyze' to begin analysis")
                with gr.Column():
                    scan_output = gr.Markdown("Scan details will appear here")
            
            with gr.Row():
                json_output = gr.JSON(label="📊 Raw Scan Data (JSON)", visible=True)
            
            # Connect scan button to function with LLM interpretation
            def scan_and_interpret(file_path):
                """Scan file and get LLM interpretation of results"""
                if not file_path:
                    return "❌ Please select a file", "No file selected", {}
                
                # Get scan results
                markdown_result, json_result = scan_file_with_virustotal(file_path)
                
                # Get LLM interpretation of the results
                if json_result and json_result.get("status") == "success":
                    llm_interpretation = interpret_scan_results_with_llm(json_result)
                    ai_output = f"## 🤖 AI Analysis\n\n{llm_interpretation}\n\n---\n\n## 📋 Technical Details\n\n{markdown_result}"
                else:
                    ai_output = markdown_result
                
                return ai_output, markdown_result, json_result
            
            scan_button.click(
                fn=scan_and_interpret,
                inputs=file_input,
                outputs=[ai_interpretation, scan_output, json_output]
            )

        # ===== TAB 3: AI Risk Hybrid Search =====
        with gr.Tab("🧠 AI Risk Search"):
            gr.Markdown("### AI Risk Database Hybrid Search")
            gr.Markdown("Lexical pre-filtering + semantic re-ranking over AI_risk_database_v4.csv")

            risk_query = gr.Textbox(
                label="Ask a question",
                placeholder="e.g., What risks involve privacy leakage?"
            )
            risk_top_k = gr.Slider(
                minimum=3,
                maximum=10,
                value=5,
                step=1,
                label="Top results"
            )
            risk_search_btn = gr.Button("🔎 Search", variant="primary")

            risk_results_md = gr.Markdown("Enter a question and click Search to see results.")
            risk_results_json = gr.JSON(label="📄 Raw Results (JSON)", visible=True)

            risk_search_btn.click(
                fn=run_ai_risk_search,
                inputs=[risk_query, risk_top_k],
                outputs=[risk_results_md, risk_results_json]
            )

    landing_search_btn.click(
        fn=landing_ai_risk_search,
        inputs=[landing_query, landing_top_k],
        outputs=[landing_output, risk_query, risk_results_md, risk_results_json]
    )

print("✅ Chatbot interface with LLM-powered file scanner created")

✅ Chatbot interface with LLM-powered file scanner created


In [10]:
## VirusTotal File Scanner Integration
# Import from service_1.py (single source of truth - enterprise pattern)
import sys
sys.path.insert(0, '/Users/retlawair/Desktop/deploying-ai/05_src/assignment-2')

from service_1 import scan_file_with_details, VIRUSTOTAL_API_KEY
from service_2 import hybrid_search, format_search_results_markdown
import json

# Alias to match Gradio naming convention
scan_file_with_virustotal = scan_file_with_details

print(f"✅ VirusTotal API Key loaded: {VIRUSTOTAL_API_KEY[:3]}...")
print("✅ VirusTotal scanner imported from service_1.py")
print("✅ AI Risk hybrid search service imported from service_2.py")
print("✅ Enterprise pattern: UI layer imports from service layer")


def interpret_scan_results_with_llm(scan_json: dict) -> str:
    """
    Use the LLM to interpret and explain VirusTotal scan results to the user.
    
    Args:
        scan_json: The JSON result from VirusTotal scan
        
    Returns:
        LLM's natural language interpretation of the scan
    """
    try:
        # Validate and safely extract scan data
        if not isinstance(scan_json, dict):
            return f"⚠️ Invalid scan data type: {type(scan_json).__name__}"
        
        # Safely extract nested dicts and convert all values to strings
        file_info = scan_json.get('file_info', {})
        if not isinstance(file_info, dict):
            file_info = {}
        
        scan_info = scan_json.get('scan_info', {})
        if not isinstance(scan_info, dict):
            scan_info = {}
        
        # Get individual values, converting to string to prevent type mismatches
        try:
            file_name = str(file_info.get('name', 'Unknown')).strip() or 'Unknown'
            file_size = str(file_info.get('size_bytes', 'Unknown')).strip() or 'Unknown'
            file_hash = str(file_info.get('sha256_hash', 'Unknown')).strip() or 'Unknown'
            scan_status = str(scan_info.get('status', 'Unknown')).strip() or 'Unknown'
            scan_id = str(scan_info.get('scan_id', 'Unknown')).strip() or 'Unknown'
        except (ValueError, TypeError) as e:
            return f"⚠️ Error processing scan data: Could not parse file information"
        
        # Format scan data for LLM analysis
        try:
            scan_summary = f"""
**File Scan Summary:**
- File: {file_name}
- Size: {file_size} bytes
- SHA256: {file_hash}
- Scan Status: {scan_status}
- Scan ID: {scan_id}

Please provide a clear, concise interpretation of this file scan result for a non-technical user. 
Explain what the status means, what they should expect, and any recommended actions.
"""
        except (TypeError, ValueError) as e:
            return "⚠️ Error formatting scan summary for analysis"
        
        # Call LLM to interpret results
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a cybersecurity expert explaining file scan results in simple, non-technical language. Be clear, concise, and actionable."
                    },
                    {
                        "role": "user",
                        "content": scan_summary
                    }
                ],
                temperature=0.5,
                max_tokens=500
            )
            
            interpretation = str(response.choices[0].message.content).strip()
        except Exception as llm_error:
            return f"⚠️ Could not get LLM interpretation: {type(llm_error).__name__}"
        
        # Apply guardrails to LLM output
        try:
            is_valid, result = guardrails.validate_output(interpretation)
            return result
        except GuardrailViolation:
            return interpretation
            
    except Exception as e:
        # Safely convert exception to string
        try:
            error_msg = str(e)
        except:
            error_msg = type(e).__name__
        return f"⚠️ Could not interpret results: {error_msg}"


print("✅ LLM-powered scan result interpreter loaded")

✅ VirusTotal API configured successfully
   API Key: ea827edd89776ebcbbcf...
   Base URL: https://www.virustotal.com/api/v3
✅ AI Risk Database hybrid search service initialized
✅ VirusTotal API Key loaded: ea8...
✅ VirusTotal scanner imported from service_1.py
✅ AI Risk hybrid search service imported from service_2.py
✅ Enterprise pattern: UI layer imports from service layer
✅ LLM-powered scan result interpreter loaded


## Step 7: Launch the Application

In [ ]:
# Launch the Gradio app
print("\n" + "="*70)
print("🚀 LAUNCHING CHATBOT APPLICATION".center(70))
print("="*70)
print()
print("📱 Open your browser and navigate to: http://localhost:7860")
print()
print("✨ Features:")
print("   • Full conversation history")
print("   • Real-time responses from GPT-4")
print("   • Amazing, responsive interface")
print("   • One-click chat management")
print()
print("⏹️  Press Ctrl+C in terminal to stop")
print()
print("="*70 + "\n")

# Launch
demo.launch(
    share=False,
    server_name="127.0.0.1",
    server_port=None,
    show_error=True,
    quiet=True
)


                   🚀 LAUNCHING CHATBOT APPLICATION                    

📱 Open your browser and navigate to: http://localhost:7860

✨ Features:
   • Full conversation history
   • Real-time responses from GPT-4
   • Amazing, responsive interface
   • One-click chat management

⏹️  Press Ctrl+C in terminal to stop




📁 Scanning file: eicar.com.txt
   Size: 68 bytes
   SHA256: 275a021bbfb6489e54d471899f7db9d1663fc695ec2fe2a2c4538aabf651fd0f
✅ File scan initiated successfully: NDRkODg2MTJmZWE4YThmMzZkZTgyZTEyNzhhYmIwMmY6MTc3MjE2NjgxNg==


## 📋 Enterprise Guardrails System - Technical Documentation

### Overview
This chatbot implements **three-layer enterprise-grade guardrails** to ensure safe and compliant AI interactions:

### 1️⃣ **Input Validation (Hard Block)**
- **Type Checking**: Validates input is non-empty string
- **Length Limits**: 1-2000 characters (configurable)
- **Forbidden Topics**: Blocks requests mentioning:
  - Cats, Dogs, Horoscopes, Zodiac Signs, Taylor Swift
- **Pattern Detection**: Regex-based detection of:
  - Prompt injection attempts
  - System prompt exposure attempts
  - Instruction override attempts
- **Rate Limiting**: 60 requests/minute per user (hard limit)

### 2️⃣ **Output Validation (Hard Block)**
- **Type Safety**: Ensures response is valid string
- **Length Enforcement**: 1-4000 characters with auto-truncation
- **Content Filtering**: Detects forbidden topics in model output
- **System Prompt Leakage Detection**: Prevents accidental exposure of system instructions
- **Automatic Sanitization**: Truncates excessive outputs with "..." indicator

### 3️⃣ **Hard Limits (Parameter Validation)**
- **Temperature**: 0.0-2.0 (deterministic to highly creative)
- **Max Tokens**: 100-4000 per request
- **Input Length**: Maximum 2000 characters
- **Output Length**: Maximum 4000 characters
- **Request Rate**: 60 requests per minute per user

### Architecture Benefits
✅ **Scalable**: Dataclass-based configuration for easy deployment variations  
✅ **Enterprise**: Severity levels (CRITICAL/WARNING) and violation tracking  
✅ **Maintainable**: Centralized GuardrailConfig class for policy updates  
✅ **Auditable**: Violation exceptions with detailed type and reason  
✅ **Safe Defaults**: Conservative limits that can be relaxed if needed  

### Usage Examples

```python
# Check guardrail status
guardrails.get_guardrail_status()

# Modify forbidden topics (if needed)
guardrails.config.forbidden_topics.append("new_topic")

# Adjust rate limits
guardrails.config.max_requests_per_minute = 120
```

### Violation Handling
- **CRITICAL violations**: Request/response blocked, user receives ⛔ message
- **WARNING violations**: Logged but execution may continue
- All violations raise `GuardrailViolation` exception with severity/type info